<a href="https://colab.research.google.com/github/BooEiEi/Master-Project/blob/main/Hyperparameter_Lightgbm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os# os ใช้จัดการกับไฟล์ ว่าอยู่ที่ไหน
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import MinMaxScaler
from google.colab import drive #เชื่อม Google drive
drive.mount('/content/drive')
path_DF = '/content/drive/My Drive/master project/data/DF3'

Mounted at /content/drive


In [ ]:
import lightgbm
import sklearn
from lightgbm import LGBMRegressor
from sklearn.model_selection import GridSearchCV, TimeSeriesSplit
from sklearn.metrics import mean_squared_error, mean_absolute_error, root_mean_squared_error, mean_absolute_percentage_error

In [ ]:
df_META = pd.read_csv(os.path.join(path_DF, 'df_META.csv'))
df_AAPL= pd.read_csv(os.path.join(path_DF, 'df_AAPL.csv'))
df_MSFT= pd.read_csv(os.path.join(path_DF, 'df_MSFT.csv'))
df_NVDA= pd.read_csv(os.path.join(path_DF, 'df_NVDA.csv'))
df_GOOG= pd.read_csv(os.path.join(path_DF, 'df_GOOG.csv'))

In [ ]:
df_META=df_META.drop(['Unnamed: 0'], axis=1)
df_META['date'] = pd.to_datetime(df_META['date'])

In [ ]:
df_AAPL=df_AAPL.drop(['Unnamed: 0'], axis=1)
df_AAPL['date'] = pd.to_datetime(df_AAPL['date'])

In [ ]:
df_MSFT=df_MSFT.drop(['Unnamed: 0'], axis=1)
df_MSFT['date'] = pd.to_datetime(df_MSFT['date'])

In [ ]:
df_NVDA=df_NVDA.drop(['Unnamed: 0'], axis=1)
df_NVDA['date'] = pd.to_datetime(df_NVDA['date'])

In [ ]:
df_GOOG=df_GOOG.drop(['Unnamed: 0'], axis=1)
df_GOOG['date'] = pd.to_datetime(df_GOOG['date'])

In [ ]:
train_size = int(len(df_META) * 0.7)  # 70% ของ 753
test_size = len(df_META) - train_size  # 30% ที่เหลือ

# META

## Train Test

In [ ]:
train_META = df_META.iloc[:train_size]  # ข้อมูล train
test_META = df_META.iloc[train_size:]  # ข้อมูล test

### Train Varidation

In [ ]:
X_META = train_META.drop(columns=["Stockprice_META","date","RETAIL_SALES", "US_Dollar_Index", "WTI"])
y_META = train_META["Stockprice_META"]

In [ ]:
X_META

,sentiment_META,Interest,UNEMPLOYMENT,CPI,US_Treasury_Yield,US_inflation_rate,Real_GDP,S&P500,NASDAQ,BTC,Gold,lag_1,lag_3,lag_5,lag_7
0,0.406150,4.33,4.0,317.671,4.53,0.030,6001.465,6144.15,20056.25,96644.8,2936.1,694.84,668.13,673.70,668.20
1,0.536325,4.33,4.0,317.671,4.55,0.030,6001.465,6129.58,20041.26,95638.0,2949.0,703.77,683.55,657.50,658.24
2,0.399740,4.33,4.0,317.671,4.47,0.030,6001.465,6114.63,20026.77,97504.6,2900.7,716.37,694.84,668.13,673.70
3,-0.726900,4.33,4.0,317.671,4.52,0.030,6001.465,6115.07,19945.64,96610.2,2945.4,736.67,703.77,683.55,657.50
4,0.304667,4.33,4.0,317.671,4.62,0.030,6001.465,6051.97,19649.95,97881.3,2928.7,728.56,716.37,694.84,668.13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
517,0.179971,4.33,3.5,299.170,3.49,0.065,5469.175,4060.43,11512.41,23016.0,1930.0,151.74,148.97,188.77,186.06
518,0.009677,4.33,3.5,299.170,3.46,0.065,5469.175,4016.22,11313.36,23055.1,1942.6,147.30,147.06,153.12,186.53
519,0.321800,4.33,3.5,299.170,3.46,0.065,5469.175,4016.95,11334.27,22632.5,1935.4,141.50,151.74,148.97,188.77
520,-0.177067,4.33,3.5,299.170,3.52,0.065,5469.175,4019.81,11364.41,22915.5,1928.6,143.14,147.30,147.06,153.12


#Normalized

In [ ]:
scaler = MinMaxScaler(feature_range=(0, 1))

X_META = scaler.fit_transform(X_META)

In [ ]:
model = LGBMRegressor(random_state=42)

param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'min_data_in_leaf': [25, 50, 100, 150, 200],
    'n_estimators' : [100, 200, 400, 600],
    'feature_fraction': [0.5, 0.6, 0.7, 0.8, 0.9],
    'max_bin' : [50, 100, 150, 200, 400]
}

tscv = TimeSeriesSplit(n_splits=5)

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs=-1)

grid_search.fit(X_META , y_META)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

y_pred = best_model.predict(X_META)
mse = mean_squared_error(y_META, y_pred)
rmse = root_mean_squared_error(y_META, y_pred)
mae = mean_absolute_error(y_META, y_pred)
mape = mean_absolute_percentage_error(y_META, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")
print(f"Test Set Root Mean Squared Error: {rmse:.4f}")
print(f"Test Set Mean Absolute Error: {mae:.4f}")
print(f"Test Set Mean Absolute Percentage Error: {mape:.4f}")

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=25
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=25
[LightGBM] [Warning] feature_fraction is set=0.6, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.6
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000055 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1067
[LightGBM] [Info] Number of data points in the train set: 522, number of used features: 15
[LightGBM] [Info] Start training from score 408.657797
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No fur

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


#AAPL

In [ ]:
train_AAPL = df_AAPL.iloc[:train_size]  # ข้อมูล train
test_AAPL = df_AAPL.iloc[train_size:]  # ข้อมูล test

In [ ]:
X_AAPL = train_AAPL.drop(columns=["Stockprice_AAPL","date", "Interest", "US_Treasury_Yield", "RETAIL_SALES", "US_Dollar_Index"])
y_AAPL = train_AAPL["Stockprice_AAPL"]

In [ ]:
X_AAPL = scaler.fit_transform(X_AAPL)

In [ ]:
model = LGBMRegressor(random_state=42)

param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'min_data_in_leaf': [25, 50, 100, 150, 200],
    'n_estimators' : [100, 200, 400, 600],
    'feature_fraction': [0.5, 0.6, 0.7, 0.8, 0.9],
    'max_bin' : [50, 100, 150, 200, 400]
}

tscv = TimeSeriesSplit(n_splits=5)

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs=-1)

grid_search.fit(X_AAPL , y_AAPL)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

y_pred = best_model.predict(X_AAPL)
mse = mean_squared_error(y_AAPL, y_pred)
rmse = root_mean_squared_error(y_AAPL, y_pred)
mae = mean_absolute_error(y_AAPL, y_pred)
mape = mean_absolute_percentage_error(y_AAPL, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")
print(f"Test Set Root Mean Squared Error: {rmse:.4f}")
print(f"Test Set Mean Absolute Error: {mae:.4f}")
print(f"Test Set Mean Absolute Percentage Error: {mape:.4f}")

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=25
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=25
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000610 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 560
[LightGBM] [Info] Number of data points in the train set: 522, number of used features: 14
[LightGBM] [Info] Start training from score 194.054751
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warni

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


#MSFT

In [ ]:
train_MSFT = df_MSFT.iloc[:train_size]  # ข้อมูล train
test_MSFT = df_MSFT.iloc[train_size:]  # ข้อมูล test

In [ ]:
X_MSFT = train_MSFT.drop(columns=["Stockprice_MSFT","date","RETAIL_SALES", "US_Dollar_Index", "WTI"])
y_MSFT = train_MSFT["Stockprice_MSFT"]

In [ ]:
X_MSFT = scaler.fit_transform(X_MSFT)

In [ ]:
model = LGBMRegressor(random_state=42)

param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'min_data_in_leaf': [25, 50, 100, 150, 200],
    'n_estimators' : [100, 200, 400, 600],
    'feature_fraction': [0.5, 0.6, 0.7, 0.8, 0.9],
    'max_bin' : [50, 100, 150, 200, 400]
}

tscv = TimeSeriesSplit(n_splits=5)

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs=-1)

grid_search.fit(X_MSFT , y_MSFT)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

y_pred = best_model.predict(X_MSFT)
mse = mean_squared_error(y_MSFT, y_pred)
rmse = root_mean_squared_error(y_MSFT, y_pred)
mae = mean_absolute_error(y_MSFT, y_pred)
mape = mean_absolute_percentage_error(y_MSFT, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")
print(f"Test Set Root Mean Squared Error: {rmse:.4f}")
print(f"Test Set Mean Absolute Error: {mae:.4f}")
print(f"Test Set Mean Absolute Percentage Error: {mape:.4f}")

[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=25
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=25
[LightGBM] [Warning] feature_fraction is set=0.8, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.8
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000189 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1067
[LightGBM] [Info] Number of data points in the train set: 522, number of used features: 15
[LightGBM] [Info] Start training from score 373.791638
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warn

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


#NVDA

In [ ]:
train_NVDA = df_NVDA.iloc[:train_size]  # ข้อมูล train
test_NVDA = df_NVDA.iloc[train_size:]  # ข้อมูล test

In [ ]:
X_NVDA = train_NVDA.drop(columns=["Stockprice_NVDA","date", "Interest","RETAIL_SALES", "US_Dollar_Index"])
y_NVDA = train_NVDA["Stockprice_NVDA"]

In [ ]:
X_NVDA = scaler.fit_transform(X_NVDA)

In [ ]:
model = LGBMRegressor(random_state=42)

param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'min_data_in_leaf': [25, 50, 100, 150, 200],
    'n_estimators' : [100, 200, 400, 600],
    'feature_fraction': [0.5, 0.6, 0.7, 0.8, 0.9],
    'max_bin' : [50, 100, 150, 200, 400]
}

tscv = TimeSeriesSplit(n_splits=5)

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs=-1)

grid_search.fit(X_NVDA , y_NVDA)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

y_pred = best_model.predict(X_NVDA)
mse = mean_squared_error(y_NVDA, y_pred)
rmse = root_mean_squared_error(y_NVDA, y_pred)
mae = mean_absolute_error(y_NVDA, y_pred)
mape = mean_absolute_percentage_error(y_NVDA, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")
print(f"Test Set Root Mean Squared Error: {rmse:.4f}")
print(f"Test Set Mean Absolute Error: {mae:.4f}")
print(f"Test Set Mean Absolute Percentage Error: {mape:.4f}")

[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=25
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=25
[LightGBM] [Warning] feature_fraction is set=0.5, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.5
[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.000049 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 1160
[LightGBM] [Info] Number of data points in the train set: 522, number of used features: 15
[LightGBM] [Info] Start training from score 77.407537
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No furt

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(


#GOOG

In [ ]:
train_GOOG = df_GOOG.iloc[:train_size]  # ข้อมูล train
test_GOOG = df_GOOG.iloc[train_size:]  # ข้อมูล test

In [ ]:
X_GOOG = train_GOOG.drop(columns=["Stockprice_GOOGL","date", "Interest", "RETAIL_SALES", "US_Dollar_Index", "WTI"])
y_GOOG = train_GOOG["Stockprice_GOOGL"]

In [ ]:
X_GOOG = scaler.fit_transform(X_GOOG)

In [ ]:
model = LGBMRegressor(random_state=42)

param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'max_depth': [3, 5, 7, 9],
    'min_data_in_leaf': [25, 50, 100, 150, 200],
    'n_estimators' : [100, 200, 400, 600],
    'feature_fraction': [0.5, 0.6, 0.7, 0.8, 0.9],
    'max_bin' : [50, 100, 150, 200, 400]
}

tscv = TimeSeriesSplit(n_splits=5)

grid_search = GridSearchCV(estimator=model, param_grid=param_grid, cv=tscv, scoring='neg_mean_squared_error', n_jobs=-1)

grid_search.fit(X_GOOG , y_GOOG)

best_model = grid_search.best_estimator_
best_params = grid_search.best_params_

y_pred = best_model.predict(X_GOOG)
mse = mean_squared_error(y_GOOG, y_pred)
rmse = root_mean_squared_error(y_GOOG, y_pred)
mae = mean_absolute_error(y_GOOG, y_pred)
mape = mean_absolute_percentage_error(y_GOOG, y_pred)
print(f"Best Hyperparameters: {best_params}")
print(f"Test Set Mean Squared Error: {mse:.4f}")
print(f"Test Set Root Mean Squared Error: {rmse:.4f}")
print(f"Test Set Mean Absolute Error: {mae:.4f}")
print(f"Test Set Mean Absolute Percentage Error: {mape:.4f}")

[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=25
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Warning] min_data_in_leaf is set=25, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=25
[LightGBM] [Warning] feature_fraction is set=0.9, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.9
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000158 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1060
[LightGBM] [Info] Number of data points in the train set: 522, number of used features: 14
[LightGBM] [Info] Start training from score 145.745814
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warn

/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
